In [1]:
from tdc.generation import MolGen
from tdc.single_pred import ADME
from tdc.single_pred import Tox
from tdc.utils import retrieve_label_name_list
import pandas as pd

path = "/home/bhux/workplace/drug_discovery/data"

def get_splits(data): 
    split = data.get_split()
    split = [split['train'], split['valid'], split['test']]
    split = pd.concat(split)
    return split

adme_names = [
    'Caco2_Wang',   # 906
    'Lipophilicity_AstraZeneca',
    'Solubility_AqSolDB',
    'HydrationFreeEnergy_FreeSolv',
    'PPBR_AZ',
    'VDss_Lombardo',
    'Half_Life_Obach',
    'Clearance_Hepatocyte_AZ',
    'Clearance_Microsome_AZ',
]

tox_names = [
    'LD50_Zhu',
]

data_dfs = {}
for name in adme_names:
    data = ADME(name = name, path=path)
    data_dfs[name] = get_splits(data)

data = Tox(name = 'LD50_Zhu', path=path)
data_dfs['LD50_Zhu'] = get_splits(data)

label_list = retrieve_label_name_list('herg_central')
for lname in label_list[:-1]: # no inhib
    data = Tox(name = 'herg_central', label_name = lname)
    data_dfs['herg_central_'+lname] = get_splits(data)

print(data_dfs.keys())

# categorical adme 
cat_adme_names = [
    # 'PAMPA_NCATS',
    # 'HIA_Hou',
    # 'Pgp_Broccatelli',
    # 'Bioavailability_Ma',
    # 'BBB_Martins',
    # 'CYP2C19_Veith',
    # 'CYP2D6_Veith',
    # 'CYP3A4_Veith',
    # 'CYP1A2_Veith',
    # 'CYP2C9_Veith',
    # 'CYP2C9_Substrate_CarbonMangels',
    # 'CYP2D6_Substrate_CarbonMangels',
    # 'CYP3A4_Substrate_CarbonMangels',
]

for name in cat_adme_names:
    data = ADME(name = name, path=path)
    data_dfs[name] = get_splits(data)

# cat_tox_names = [
#     'AMES',
#     'DILI',
#     'Skin Reaction',
#     'Carcinogens_Lagunin',
#     'ClinTox',
# ]

# for name in cat_tox_names:
#     data = Tox(name = name, path=path)
#     data_dfs[name] = get_splits(data)

# # Tox21 and ToxCast
# name = 'Tox21'
# label_list = retrieve_label_name_list(name)
# for lname in label_list: # no inhib
#     data = Tox(name = name, label_name = lname)
#     data_dfs[name+lname] = get_splits(data)

# name = 'Toxcast'
# label_list = retrieve_label_name_list(name)
# for lname in label_list[:10]: # no inhib
#     data = Tox(name = name, label_name = lname)
#     data_dfs[name+lname] = get_splits(data)

Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


dict_keys(['Caco2_Wang', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB', 'HydrationFreeEnergy_FreeSolv', 'PPBR_AZ', 'VDss_Lombardo', 'Half_Life_Obach', 'Clearance_Hepatocyte_AZ', 'Clearance_Microsome_AZ', 'LD50_Zhu', 'herg_central_hERG_at_1uM', 'herg_central_hERG_at_10uM'])


In [2]:
import numpy as np

# Populate smiles
smiles = []
for _, df in data_dfs.items():
    smiles.extend(list(df['Drug']))
smiles = list(set(smiles))
print(len(smiles))

# Build dataset
lengths = []
dataset = {'Drug': smiles}
for k, df in data_dfs.items():
    df = {v['Drug']: v['Y'] for v in df.to_dict(orient='records')}
    dataset[k] = []
    for smile in smiles:
        dataset[k].append(df.get(smile, np.nan))
    lengths.append(len(dataset[k])-sum(np.isnan(np.array(dataset[k]))))

print(dataset.keys())

df = pd.DataFrame(data=dataset)

display(df[:10])



327533
dict_keys(['Drug', 'Caco2_Wang', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB', 'HydrationFreeEnergy_FreeSolv', 'PPBR_AZ', 'VDss_Lombardo', 'Half_Life_Obach', 'Clearance_Hepatocyte_AZ', 'Clearance_Microsome_AZ', 'LD50_Zhu', 'herg_central_hERG_at_1uM', 'herg_central_hERG_at_10uM'])


,Drug,Caco2_Wang,Lipophilicity_AstraZeneca,Solubility_AqSolDB,HydrationFreeEnergy_FreeSolv,PPBR_AZ,VDss_Lombardo,Half_Life_Obach,Clearance_Hepatocyte_AZ,Clearance_Microsome_AZ,LD50_Zhu,herg_central_hERG_at_1uM,herg_central_hERG_at_10uM
0,CCOc1ccc(OCCCC(=O)N(CC)CC(=O)Nc2c(F)cccc2F)cc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.1155,-5.99112
1,Oc1ccc(Br)cc1-c1cc(-c2ccccc2)n[nH]1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.3017,-3.11242
2,Cc1noc(C)c1S(=O)(=O)N1CCC(C(=O)Nc2ccccn2)CC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0207,-14.49412
3,CCc1nnc(NC(=O)c2ccc(-c3cccc(C(F)(F)F)c3)o2)s1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7267,22.04728
4,O=S(=O)(Cc1ccccc1)N1CCN(Cc2ccc3c(c2)OCO3)CC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0026,-6.27632
5,Cc1c(-c2ccccc2)oc2c(C(=O)OCCN3CCCCC3)cccc2c1=O,NaN,3.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,COc1ccc(CNC(=O)CSc2nnnn2-c2ccc(O)cc2)cc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8222,-11.19322
7,O=C(O)CC(NNC(=O)c1ccccc1)C(=O)Nc1ccc([N+](=O)[...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.3764,1.38238
8,CCOC(=O)c1nnc(-c2ccccc2)nc1Sc1ccccc1Cl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.3281,8.65718
9,CCCC(=O)C1=C(N2CCCC2)CC(C)(C)C(C(=O)OC)C1=O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.1432,7.44698


In [3]:
from collections import Counter

combinations = []
for row in df.drop(labels=['Drug', 'herg_central_hERG_at_10uM'], axis=1).iterrows():
    combinations.append("".join([str(int(np.isnan(v) == False)) for _,v in list(row)[1].items()]))

uc = Counter(combinations)
print(uc.keys())
print(uc.values())

dict_keys(['00000000001', '01000000000', '00100000000', '00000000010', '00110000000', '01000000001', '00001000001', '01000000100', '00000100000', '11100000011', '00100000010', '10000000010', '01001000000', '01100000000', '00100000001', '10000000000', '00000010000', '00110000010', '00001000000', '00000000011', '00100000011', '01001000001', '01001001100', '00000001100', '01001000100', '01100000011', '11000010000', '10100010001', '01100111000', '00000100001', '01000001100', '00001001100', '01001001000', '01001001101', '00000000100', '01000001000', '00010000000', '00001000100', '00001001001', '10100000000', '00010000001', '00101011111', '11001011000', '00000001000', '10000000001', '00000010001', '01100010000', '11001011100', '10110000000', '01000000010', '11001110011', '10000100000', '00100110001', '00001001000', '10000110011', '00100010011', '00010000010', '00100110011', '00001010001', '00110000011', '11100000000', '00000100010', '01000010010', '11101001111', '11000001001', '01000010000',

In [4]:
cbool = np.array(combinations) != '0'*10 + '1' + '0'*(len(list(uc.keys())[0])-11) #'00000000001000000000000000000000000000000'
inc = np.random.choice([i for i, x in enumerate(cbool) if x == False], 7900, replace=False)
cbool = [True if i in inc else v for i,v in enumerate(cbool)]

fcomb = np.array(combinations)[cbool]
fdf = df[cbool]

print(len(fdf.index))

30000


In [5]:
def unison_shuffled_copies(a, b):
    #assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], p

rcomb, perm = unison_shuffled_copies(fcomb, fdf.iloc)
lengths[10] = 7900
del lengths[11]
valCount = np.array(lengths)*0.1

trIdx = []
valIdx = []
for c, p in zip(rcomb, perm):
    inc = False
    for i, j in enumerate(list(c)):
        if j == "1" and valCount[i] > 0:
            valCount[i] -= 1
            inc = True
    if inc:
        valIdx.append(p)
    else:
        trIdx.append(p)

print(len(trIdx))
print(len(valIdx))

27067
2933


In [6]:
trdf = fdf.iloc[trIdx]
valdf = fdf.iloc[valIdx]

In [7]:
from transformers import AutoTokenizer, AutoModel, T5EncoderModel, AutoModelForMaskedLM
import torch
from tqdm import tqdm

model = "sagawa/PubChem-10m-deberta"
# "sagawa/PubChem-10m-deberta"
# "seyonec/ChemBERTa-zinc-base-v1"
# "DeepChem/ChemBERTa-10M-MLM"
# "sagawa/PubChem-10m-t5-v2"

# "ncfrey/ChemGPT-4.7M"
# "RaphaelMourad/Mistral-Chem-v1-417M"
# "jonghyunlee/ChemBERT_ChEMBL_pretrained"

# Initialize tokenizer and model for drug embeddings
drug_tokenizer = AutoTokenizer.from_pretrained(model)
drug_model = AutoModel.from_pretrained(model)

drug_model.to('cuda')

# Function to get drug embeddings from SMILES strings
def get_drug_embeddings(smiles_list, tokenizer, model, batch_size=64):
    embeddings = []
    for i in tqdm(range(0, len(smiles_list), batch_size)):
        batch = smiles_list[i:i+batch_size]
        encoding = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        input_ids = encoding["input_ids"].to('cuda')
        attention_mask = encoding["attention_mask"].to('cuda')  # Get the attention mask
        
        with torch.no_grad():
            # Pass both input_ids and attention_mask to the model
            output = model(input_ids, attention_mask=attention_mask)
            # You can use either the last hidden state or the pooled output
            # depending on your requirements. Here we're using the mean of the last hidden state.
            embedding = output.last_hidden_state.mean(1)
            embeddings.extend(embedding.cpu().numpy())
            
    return embeddings

In [8]:
# Save train
drug_embeddings_split_kd = get_drug_embeddings(list(fdf['Drug']), drug_tokenizer, drug_model)
print(len(drug_embeddings_split_kd))
np.save("./data/xtended_emb_all_deberta_pubchem", np.array(drug_embeddings_split_kd))
fdf.to_csv("./data/xtended_data_all.csv", index=False)

100%|██████████| 469/469 [00:43<00:00, 10.76it/s]


30000


In [9]:
drug_embeddings_split_kd = get_drug_embeddings(list(valdf['Drug']), drug_tokenizer, drug_model)
print(len(drug_embeddings_split_kd))
np.save("./data/xtended_emb_val_deberta_pubchem", np.array(drug_embeddings_split_kd))
valdf.to_csv("./data/xtended_data_val.csv", index=False)

100%|██████████| 46/46 [00:03<00:00, 11.61it/s]

2933
